# Librerias a usar para el parser

In [173]:
from typing import Union, Optional, List, Dict, Iterator, Tuple, Literal
from dataclasses import dataclass
from pathlib import Path
from datetime import date
import pandas as pd
import math as mh
import re

# Mejoras de constantes

In [124]:
# Constantes para nombres de grupos de la regex
G_EST      = 'est'
G_VAL_571  = 'val_571'
G_VAL_572  = 'val_572'
G_VAL_573  = 'val_573'
G_VAL_574  = 'val_574'
G_VAL_83   = 'val_83'

escala: Dict[str, float] = {'0': 1e3, '6': 1e4, '8': 1e5}

@dataclass
class Medida:
    orientation: Literal["ini", "fin"]
    dif_dist: Optional[float] = None
    total_dist: Optional[float] = None
    dif_height: Optional[float] = None
    med_ini: Optional[float] = None
    # metadatos útiles para trazabilidad
    est: Optional[str] = None
    file: Optional[str] = None

# Paths y REGEX

In [2]:
# Paths y expresiones

PATH = Path("G:\\Otros ordenadores\\Mi PC\\IGAC\\archivosPrueba\\Archivos_Pruebas\\L5_GSI_1")
L5Crudos = PATH / 'CRUDOS_L5'
L5Orden = PATH / 'Orden_L5' / 'L5_Principio_Linea;Fin_linea.txt'
crudos = [p for p in L5Crudos.iterdir() if p.is_file() is True]

# BLOCK: Patron que separa los bloques
BLOCK_RE = re.compile(r'^\s*(?P<grupo>[^+\s]+)\+[^?]*\?.*?(?P<codigo>[1-4]|10)\b')

# VERT: Patron que estandariza los vertices XXXX-XXXX-09
VERT_RE = re.compile(r'^([A-Z0-9]+)([A-Z]{2,4})(\d{1,2})$')

# FECHA6: Extrae fechas
FECHA6_RE = re.compile(r'(\d{6})')

# INI: siguiente línea del bloque. Se exige 83..[068]<valor> y se toma como ini si <valor> == 0
INI_RE = re.compile(r"""
    (?P<est>[A-Z0-9]+)\s+
    (?P<idx_83>83)\.\.(?P<p5_83>[25])(?P<p6_83>[068])(?P<val_83>[+-]?\d+)
""", re.X)

# FIN: línea BF completa (573, 574, 83)
FIN_RE_BF = re.compile(r"""
    (?P<est>[A-Z0-9]+)\s+
    (?P<idx_573>573)\.\.(?P<p6_573>[068])(?P<val_573>[+-]?\d+)\s+
    (?P<idx_574>574)\.\.(?P<p6_574>[068])(?P<val_574>[+-]?\d+)\s+
    (?P<idx_83>83)\.\.(?P<p5_83>[0125])(?P<p6_83>[068])(?P<val_83>[+-]?\d+)
""", re.X)

# Estandarizador de Nomenclaturas

In [3]:
# Función que estandariza las nomenclaturas
def vert_std(codigo:str) -> str:
    code_mayusc = codigo.replace("-", "").replace(" ", "").upper()
    m = VERT_RE.match(code_mayusc)
    if m:
        g1, g2, g3 = m.groups()
        return f"{g1}-{g2}-{g3}"
    
    return codigo

print(vert_std('A50BE2'))

A50-BE-2


# Clasificador de Orden

In [38]:
import json

# Función que ordena el archivo orden
def clasi_orden(forden:Path, headline:bool = True) -> List[Dict[str,str]]:
    output = []

    if headline:
        with open(forden, "r", encoding="utf-8") as f:
            # suponemos primera línea encabezado
            for i, linea in enumerate(f.readlines()[1:]):
                linea = linea.strip()
                if not linea or ";" not in linea:
                    continue
                a, b = linea.split(";")
                a = vert_std(a)
                b = vert_std(b)
                if i % 2 == 0:
                    output.append({"ini":a,"fin":b,"sentido":"I"})
                else:
                    output.append({"ini":a,"fin":b,"sentido":"R"})
    else:
        with open(forden, "r", encoding="utf-8") as f:
            # suponemos primera línea encabezado
            for i, linea in enumerate(f.readlines()):
                linea = linea.strip()
                if not linea or ";" not in linea:
                    continue
                a, b = linea.split(";")
                a = vert_std(a)
                b = vert_std(b)
                if i % 2 == 0:
                    data.append({"ini":a,"fin":b,"sentido":"I"})
                else:
                    data.append({"ini":a,"fin":b,"sentido":"R"})

    with open("../LeicaGSI_LS10_Parser/data/orden_niv.json", "w") as f:
        json.dump(output, f, indent=2)

    return output

data = clasi_orden(L5Orden)
pd.DataFrame(data)

,ini,fin,sentido
0,B70-CW-7,23660003,I
1,23660003,B70-CW-7,R
2,23660003,A2-CB-3,I
3,A2-CB-3,23660003,R
4,A2-CB-3,A3-CB-3,I
...,...,...,...
165,A32-BE-2,A31-BE-2,R
166,A32-BE-2,A33-BE-2,I
167,A33-BE-2,A32-BE-2,R
168,A33-BE-2,B33-CW-7,I


# Generador de Fechas

In [39]:
def ext_fecha(fname:str):
    m = FECHA6_RE.search(fname)
    if not m:
        return None
    
    y, mth, d = int(m.group(1)[:2]), int(m.group(1)[2:4]), int(m.group(1)[4:6])

    # regla para siglo: 00–49 → 2000s, 50–99 → 1900s
    year = 2000 + y if y < 50 else 1900 + y

    try:
        return date(year, mth, d).isoformat()
    except ValueError:
        return None
    
print(ext_fecha('100517'))

2010-05-17


# Calculo colimación

In [219]:
def colimacion(lineas,file):
    """
    La función colimación tiene como proposito calcular el error de colimación en metros para posteriormente
    ser tenido en cuenta en el ajuste. La metodología con la que es calculada es la mencionada en el manual de 
    Leica para equipos LS10/LS15.
    
    Parametros:
        - lineas: Es el conjunto de lineas del archivo para que la misma función examine si hay 
    """
    patron_colimacion = r'^(\d{6})\+(\S{8,16})\s+(32\.\.\.[068])([+-]\S{8,16})\s(\S{6})([+-]\S{8,16})'
    colimacion = []
    tipo_gsi = None
    for linea in lineas:
        linea = linea.strip()
        try:
            pat_col = re.search(patron_colimacion,linea) # patron de colimación
            if pat_col is not None:
                try:
                    tipo_gsi = len(pat_col.group(2)) # Obtiene el tipo de archivo gsi que se esta manejando
                    data_col = ['A1','B1','B2','A2']
                    if pat_col.group(2).lstrip('0') in data_col:
                        try:
                            conversor = escala.get(pat_col.group(3)[-1])
                            col_data = {
                                "id_colimacion": pat_col.group(2).lstrip('0'),
                                "alt.obs" : float(pat_col.group(6))/conversor,
                                "dist.hor": float(pat_col.group(4))/conversor
                            }
                            colimacion.append(col_data)

                        except ValueError as e:
                            print(f"Error al obtener los datos de colimación en: {Path(file)}")
                            return None
                    else:
                        pass
                except Exception as e:
                    print(f"El error {e} se presenta en el archivo: {Path(file)}")
            else:
                pass
        except Exception as e:
            print(f"El error {e} se presenta en el archivo: {Path(file)}")

    try:
        output = mh.atan(((colimacion[0]['alt.obs'] - colimacion[1]['alt.obs']) + 
                            (colimacion[2]['alt.obs'] - colimacion[3]['alt.obs'])) / 
                            ((colimacion[0]['dist.hor'] - colimacion[1]['dist.hor']) + 
                            (colimacion[2]['dist.hor'] - colimacion[3]['dist.hor']))) * (6378137/3600)
        return [round(output,5),tipo_gsi]
    except Exception as e:
                        print(f"El error: {e} \nSe encuentra en el archivo: {Path(file)}")
                        return None

# Parser de estaciones de Inicio

In [220]:
import math as mh

archivos = sorted([p for p in L5Crudos.iterdir() if p.is_file() is True])
archivo_prueba = archivos[10]

with open(archivo_prueba,"r") as file:
    lineas = file.readlines()
    col = colimacion(lineas,archivo_prueba)
    if col:
        print(f"Error de colimación del archivo {archivo_prueba.name}: {col[0]} m\nEl tipo de archivo GSI es: {col[1]} bits")
    for i, linea in enumerate(lineas):
        linea = linea.strip()


Error de colimación del archivo 100524DL.GSI: 0.00887 m
El tipo de archivo GSI es: 8 bits


# Manejo de clases

In [128]:
linea = "110042+23001003 573..6+00005738 574..6+16214182 83..26-00002484"
linea2 = "110009+00000001 573..6-00010980 574..6+00321848 83..26+00012290"
linea3 = "110083+00A42CB3 573..6-00047557 574..6+17188834 83..26+00022618"
linea4 = "110121+00000012 573..6-00020700 574..6+15287876 83..26+00008417"
linea5 = "110056+50D-CU-1 571.28+00000006 572.28-00000052 573..8-00090868 574..8+84462336 83..28+00589637"

RE_PRUEBA = re.compile(r'''
^
(\d{6})\+                                          # 1: seis dígitos y '+'

(?P<id_vert>                                       # 2: token después del '+'
    (?:                                            #    alternancia excluyente
        (?=\d{8}(?=\s|$))(?!\d{0,4}0000)\d{8}      #    numérico: 8 dígitos y sin '0000'
      | (?=[A-Za-z0-9-]{3,8}(?=\s|$))               #    alfanumérico: 3–8, termina en espacio/fin
        (?=[A-Za-z0-9-]*[A-Za-z])                   #    contiene ≥1 letra
        [A-Za-z0-9-]{3,8}
    )
)
.*?                                                # separador
# ---- Bloque 573 (exactamente 15 caracteres) ----
(?P<b573>573\.\.[068](?P<med_573>[+-]\d{8}))\s+

# ---- Bloque 574 (exactamente 15 caracteres) ----
(?P<b574>574\.\.[068](?P<med_574>[+-]\d{8}))\s+

# ---- Bloque 83 (exactamente 15 caracteres) ----
(?P<b83>83\.\.(?P<corr_83>\d)?(?P<esc_83>\d)(?P<med_83>[+-]\d{8}))
''', re.VERBOSE)

ver = RE_PRUEBA.search(linea3)
if ver is not None:
    print(ver.groupdict())
else:
    print(None)


{'id_vert': '00A42CB3', 'b573': '573..6-00047557', 'med_573': '-00047557', 'b574': '574..6+17188834', 'med_574': '+17188834', 'b83': '83..26+00022618', 'corr_83': '2', 'esc_83': '6', 'med_83': '+00022618'}
